In [0]:
!pip install toml

In [0]:
dbutils.library.restartPython()

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import toml
import numpy as np
import joblib
import psycopg2

# 1️⃣ Leer secretos desde secrets.toml
secrets = toml.load("secrets.toml")["postgres"]
USER = secrets["USER"]
PASSWORD = secrets["PASSWORD"]
HOST = secrets["HOST"]
PORT = secrets["PORT"]
DBNAME = secrets["DBNAME"]

# 2️⃣ Cargar modelo y scaler
model = joblib.load("components/diabetes_model.pkl")
scaler = joblib.load("components/diabetes_scaler.pkl")
with open("components/diabetes_model_info.pkl", "rb") as f:
    model_info = joblib.load(f)

feature_names = model_info["feature_names"]

# 3️⃣ Generar 20 registros aleatorios
# Valores basados en rangos aproximados de las features del dataset
np.random.seed(42)
data = []
for _ in range(10):
    record = [
        float(np.random.normal(0, 0.1)) for _ in feature_names
    ]
    data.append(record)

# 4️⃣ Predecir y guardar en DB
try:
    connection = psycopg2.connect(
        user=USER,
        password=PASSWORD,
        host=HOST,
        port=PORT,
        dbname=DBNAME
    )
    cursor = connection.cursor()

    for record in data:
        features_scaled = scaler.transform([record])
        prediction = float(model.predict(features_scaled)[0])

        # Query SQL
        columns = ", ".join(feature_names)
        placeholders = ", ".join(["%s"] * (len(record) + 1))  # +1 for prediction
        sql = f"INSERT INTO pc_ml_diabetes ({columns}, prediction) VALUES ({placeholders})"

        # Insert into DB
        cursor.execute(sql, record + [prediction])

    connection.commit()
    cursor.close()
    connection.close()
    print("✅ 10 registros generados, predichos y guardados en la base de datos.")

except Exception as e:
    print(f"❌ Error al guardar en la base de datos: {e}")
